# Initial Data Setup

In [2]:
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import tensorflow as tf
import keras_tuner as kt

In [3]:
# Files to Import
all_nba_file = Path('Resources/End of Season Teams.csv')
all_star_file = Path('Resources/All-Star Selections.csv')
stats_file = Path('Resources/Player Per Game.csv')

# Set up DataFrames
all_nba_df = pd.read_csv(all_nba_file)
all_star_df = pd.read_csv(all_star_file)
stats_df = pd.read_csv(stats_file)

In [4]:
# Review DataFrames
all_nba_df.head()

,season,lg,type,number_tm,player,position,seas_id,player_id,birth_year,tm,age
0,2022,NBA,All-Defense,1st,Giannis Antetokounmpo,NaN,29918,4164,NaN,MIL,27
1,2022,NBA,All-Defense,1st,Jaren Jackson Jr.,NaN,29991,4632,NaN,MEM,22
2,2022,NBA,All-Defense,1st,Marcus Smart,NaN,30183,4275,NaN,BOS,27
3,2022,NBA,All-Defense,1st,Mikal Bridges,NaN,30205,4658,NaN,PHO,25
4,2022,NBA,All-Defense,1st,Rudy Gobert,NaN,30313,4199,NaN,UTA,29


In [5]:
all_star_df.head()

,player,team,lg,season,replaced
0,Bam Adebayo,East,NBA,2024,False
1,Giannis Antetokounmpo,East,NBA,2024,False
2,Paolo Banchero,East,NBA,2024,False
3,Scottie Barnes,East,NBA,2024,False
4,Jaylen Brown,East,NBA,2024,False


In [6]:
stats_df.head()

,seas_id,season,player_id,player,birth_year,pos,age,experience,lg,tm,...,ft_percent,orb_per_game,drb_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game
0,31136,2024,5025,A.J. Green,NaN,SG,24.0,2,NBA,MIL,...,1.000,0.2,0.9,1.0,0.5,0.1,0.1,0.1,0.9,4.3
1,31137,2024,5026,A.J. Lawson,NaN,SG,23.0,2,NBA,DAL,...,0.632,0.4,0.8,1.2,0.5,0.3,0.1,0.4,0.7,3.8
2,31138,2024,5027,AJ Griffin,NaN,SF,20.0,2,NBA,ATL,...,1.000,0.1,0.7,0.8,0.2,0.1,0.1,0.3,0.3,2.1
3,31139,2024,4219,Aaron Gordon,NaN,PF,28.0,10,NBA,DEN,...,0.652,2.4,4.1,6.5,3.2,0.9,0.7,1.5,1.9,13.9
4,31140,2024,4582,Aaron Holiday,NaN,PG,27.0,6,NBA,HOU,...,0.889,0.3,1.4,1.7,1.8,0.5,0.1,0.8,1.6,7.0


# Data Cleaning

In [7]:
# Dropping unneeded columns
all_nba_clean = all_nba_df.drop(columns=['lg','number_tm','position','seas_id','player_id','birth_year','tm','age'])

# Getting dummy columns and combining the players seasons so that their multiple awards are shown
all_nba_pivot = pd.pivot_table(all_nba_clean, index=['season','player'], columns='type', aggfunc=lambda x:'Yes').fillna('No')

# Dropping the old awards for defunct leagues
all_nba_pivot_clean = all_nba_pivot.drop(columns=['All-ABA','All-BAA'])
all_nba_pivot_clean.tail(25)

type                         All-Defense All-NBA All-Rookie
season player                                              
2022   Evan Mobley                    No      No        Yes
       Franz Wagner                   No      No        Yes
       Giannis Antetokounmpo         Yes     Yes         No
       Herbert Jones                  No      No        Yes
       Ja Morant                      No     Yes         No
       Jalen Green                    No      No        Yes
       Jaren Jackson Jr.             Yes      No         No
       Jayson Tatum                   No     Yes         No
       Joel Embiid                    No     Yes         No
       Josh Giddey                    No      No        Yes
       Jrue Holiday                  Yes      No         No
       Karl-Anthony Towns             No     Yes         No
       Kevin Durant                   No     Yes         No
       LeBron James                   No     Yes         No
       Luka Dončić                    No     Yes         No
       Marcus Smart                  Yes      No         No
       Matisse Thybulle              Yes      No         No
       Mikal Bridges                 Yes      No         No
       Nikola Jokić                   No     Yes         No
       Pascal Siakam                  No     Yes         No
       Robert Williams               Yes      No         No
       Rudy Gobert                   Yes      No         No
       Scottie Barnes                 No      No        Yes
       Stephen Curry                  No     Yes         No
       Trae Young                     No     Yes         No

In [8]:
# Dropping unneeded columns
all_star_clean = all_star_df.drop(columns=['team','lg','replaced'])
all_star_clean['All Star'] = 'Yes'
all_star_clean.head()

,player,season,All Star
0,Bam Adebayo,2024,Yes
1,Giannis Antetokounmpo,2024,Yes
2,Paolo Banchero,2024,Yes
3,Scottie Barnes,2024,Yes
4,Jaylen Brown,2024,Yes


In [63]:
# Merging
first_merged_df = pd.merge(stats_df, all_nba_pivot_clean, on=['season','player'], how='left')
merged_df = pd.merge(first_merged_df, all_star_clean, on=['season','player'], how='left')

merged_df = merged_df[merged_df['lg'] == 'NBA']
merged_df.drop(columns=['player','pos','seas_id','player_id','birth_year','lg','tm','fg_per_game','fga_per_game','x3p_per_game','x2p_per_game',
'x2pa_per_game','x2p_percent','ft_per_game'], inplace=True)

merged_df.head()

,season,age,experience,g,gs,mp_per_game,fg_percent,x3pa_per_game,x3p_percent,e_fg_percent,...,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,All-Defense,All-NBA,All-Rookie,All Star
0,2024,24.0,2,39,0.0,9.2,0.438,2.8,0.423,0.621,...,0.5,0.1,0.1,0.1,0.9,4.3,NaN,NaN,NaN,NaN
1,2024,23.0,2,28,0.0,8.3,0.471,1.4,0.325,0.547,...,0.5,0.3,0.1,0.4,0.7,3.8,NaN,NaN,NaN,NaN
2,2024,20.0,2,18,0.0,7.3,0.289,1.8,0.273,0.389,...,0.2,0.1,0.1,0.3,0.3,2.1,NaN,NaN,NaN,NaN
3,2024,28.0,10,54,54.0,31.5,0.557,1.8,0.293,0.585,...,3.2,0.9,0.7,1.5,1.9,13.9,NaN,NaN,NaN,NaN
4,2024,27.0,6,56,1.0,17.3,0.455,3.0,0.410,0.565,...,1.8,0.5,0.1,0.8,1.6,7.0,NaN,NaN,NaN,NaN


# Creating the All Star Prediction Model

In [94]:
# Setting up the data sets for each awards models
all_star_data = merged_df.copy()
all_star_data['All Star'].fillna('No', inplace=True)
y = all_star_data['All Star']
X = all_star_data.drop(columns=['season','All-Defense','All-NBA','All-Rookie','All Star']).fillna(0)

# Mapping dictionary
mapping = {'Yes': 1, 'No': 0}
y = y.map(mapping)

# Splitting into training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20)

# Scaling the x data
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
print(len(y))

29594


In [89]:
# Define the second model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = 20
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_52 (Dense)            (None, 8)                 168       
                                                                 
 dense_53 (Dense)            (None, 5)                 45        
                                                                 
 dense_54 (Dense)            (None, 5)                 30        
                                                                 
 dense_55 (Dense)            (None, 1)                 6         
                                                                 
Total params: 249
Trainable params: 249
Non-trainable params: 0
_________________________________________________________________


In [90]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [92]:
# Fit the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
694/694 [==============================] - 0s 468us/step - loss: 0.0849 - accuracy: 0.9640
Epoch 2/100
694/694 [==============================] - 0s 453us/step - loss: 0.0845 - accuracy: 0.9631
Epoch 3/100
694/694 [==============================] - 0s 458us/step - loss: 0.0844 - accuracy: 0.9645
Epoch 4/100
694/694 [==============================] - 0s 456us/step - loss: 0.0839 - accuracy: 0.9635
Epoch 5/100
694/694 [==============================] - 0s 452us/step - loss: 0.0838 - accuracy: 0.9639
Epoch 6/100
694/694 [==============================] - 0s 451us/step - loss: 0.0833 - accuracy: 0.9639
Epoch 7/100
694/694 [==============================] - 0s 458us/step - loss: 0.0833 - accuracy: 0.9646
Epoch 8/100
694/694 [==============================] - 0s 451us/step - loss: 0.0829 - accuracy: 0.9645
Epoch 9/100
694/694 [==============================] - 0s 489us/step - loss: 0.0830 - accuracy: 0.9638
Epoch 10/100
694/694 [==============================] - 0s 470us/step - l

In [93]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

232/232 - 0s - loss: 0.0808 - accuracy: 0.9642 - 151ms/epoch - 649us/step
Loss: 0.08079398423433304, Accuracy: 0.9641843438148499


# Setting up All-NBA Prediction Model

In [101]:
# Setting up the data sets for each awards models
all_nba_data = merged_df.copy().drop(columns=['All-Defense','All Star','All-Rookie'])
all_nba_data = all_nba_data[(all_nba_data['season'] != 2023) & (all_nba_data['season'] != 2024)]
all_nba_data['All-NBA'].fillna('No', inplace=True)
y = all_nba_data['All-NBA']
X = all_nba_data.drop(columns=['All-NBA','season']).fillna(0)

# Mapping dictionary
mapping = {'Yes': 1, 'No': 0}
y = y.map(mapping)

# Splitting into training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20)

# Scaling the x data
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [102]:
# Define the second model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = 20
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_56 (Dense)            (None, 8)                 168       
                                                                 
 dense_57 (Dense)            (None, 5)                 45        
                                                                 
 dense_58 (Dense)            (None, 5)                 30        
                                                                 
 dense_59 (Dense)            (None, 1)                 6         
                                                                 
Total params: 249
Trainable params: 249
Non-trainable params: 0
_________________________________________________________________


In [103]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [104]:
# Fit the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
662/662 [==============================] - 1s 504us/step - loss: 0.2488 - accuracy: 0.9513
Epoch 2/100
662/662 [==============================] - 0s 453us/step - loss: 0.0806 - accuracy: 0.9681
Epoch 3/100
662/662 [==============================] - 0s 447us/step - loss: 0.0632 - accuracy: 0.9759
Epoch 4/100
662/662 [==============================] - 0s 448us/step - loss: 0.0573 - accuracy: 0.9776
Epoch 5/100
662/662 [==============================] - 0s 454us/step - loss: 0.0544 - accuracy: 0.9789
Epoch 6/100
662/662 [==============================] - 0s 472us/step - loss: 0.0526 - accuracy: 0.9793
Epoch 7/100
662/662 [==============================] - 0s 466us/step - loss: 0.0511 - accuracy: 0.9793
Epoch 8/100
662/662 [==============================] - 0s 459us/step - loss: 0.0506 - accuracy: 0.9801
Epoch 9/100
662/662 [==============================] - 0s 469us/step - loss: 0.0494 - accuracy: 0.9806
Epoch 10/100
662/662 [==============================] - 0s 460us/step - l

In [105]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

221/221 - 0s - loss: 0.0516 - accuracy: 0.9795 - 145ms/epoch - 654us/step
Loss: 0.05164393037557602, Accuracy: 0.9794617295265198


# Setting up All-Defense Prediction Model

In [106]:
# Setting up the data sets for each awards models
all_d_data = merged_df.copy()
all_d_data = all_d_data[(all_d_data['season'] != 2023) & (all_d_data['season'] != 2024)]
all_d_data['All-Defense'].fillna('No', inplace=True)
y = all_d_data['All-Defense']
X = all_d_data.drop(columns=['All-NBA','All Star','All-Rookie','All-Defense','season']).fillna(0)

# Mapping dictionary
mapping = {'Yes': 1, 'No': 0}
y = y.map(mapping)

# Splitting into training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20)

# Scaling the x data
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [107]:
# Define the second model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = 20
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_60 (Dense)            (None, 8)                 168       
                                                                 
 dense_61 (Dense)            (None, 5)                 45        
                                                                 
 dense_62 (Dense)            (None, 5)                 30        
                                                                 
 dense_63 (Dense)            (None, 1)                 6         
                                                                 
Total params: 249
Trainable params: 249
Non-trainable params: 0
_________________________________________________________________


In [108]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [109]:
# Fit the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
662/662 [==============================] - 1s 510us/step - loss: 0.1927 - accuracy: 0.9589
Epoch 2/100
662/662 [==============================] - 0s 464us/step - loss: 0.0719 - accuracy: 0.9798
Epoch 3/100
662/662 [==============================] - 0s 457us/step - loss: 0.0648 - accuracy: 0.9800
Epoch 4/100
662/662 [==============================] - 0s 452us/step - loss: 0.0616 - accuracy: 0.9803
Epoch 5/100
662/662 [==============================] - 0s 450us/step - loss: 0.0597 - accuracy: 0.9813
Epoch 6/100
662/662 [==============================] - 0s 452us/step - loss: 0.0585 - accuracy: 0.9820
Epoch 7/100
662/662 [==============================] - 0s 473us/step - loss: 0.0577 - accuracy: 0.9820
Epoch 8/100
662/662 [==============================] - 0s 462us/step - loss: 0.0570 - accuracy: 0.9816
Epoch 9/100
662/662 [==============================] - 0s 453us/step - loss: 0.0560 - accuracy: 0.9820
Epoch 10/100
662/662 [==============================] - 0s 452us/step - l

In [110]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

221/221 - 0s - loss: 0.0557 - accuracy: 0.9819 - 146ms/epoch - 660us/step
Loss: 0.055692341178655624, Accuracy: 0.9818696975708008


# Setting up All-Rookie Prediction Model

In [111]:
# Setting up the data sets for each awards models
all_rook_data = merged_df.copy()
all_rook_data = all_rook_data[(all_rook_data['season'] != 2023) & (all_rook_data['season'] != 2024) & (all_rook_data['experience'] == 1)]
all_rook_data['All-Rookie'].fillna('No', inplace=True)
y = all_rook_data['All-Rookie']
X = all_rook_data.drop(columns=['All-NBA','All Star','All-Rookie','All-Defense','season']).fillna(0)

# Mapping dictionary
mapping = {'Yes': 1, 'No': 0}
y = y.map(mapping)

# Splitting into training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20)

# Scaling the x data
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [112]:
# Define the second model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = 20
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_64 (Dense)            (None, 8)                 168       
                                                                 
 dense_65 (Dense)            (None, 5)                 45        
                                                                 
 dense_66 (Dense)            (None, 5)                 30        
                                                                 
 dense_67 (Dense)            (None, 1)                 6         
                                                                 
Total params: 249
Trainable params: 249
Non-trainable params: 0
_________________________________________________________________


In [113]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [114]:
# Fit the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
116/116 [==============================] - 0s 537us/step - loss: 0.5523 - accuracy: 0.8660
Epoch 2/100
116/116 [==============================] - 0s 471us/step - loss: 0.2673 - accuracy: 0.9014
Epoch 3/100
116/116 [==============================] - 0s 465us/step - loss: 0.1719 - accuracy: 0.9361
Epoch 4/100
116/116 [==============================] - 0s 491us/step - loss: 0.1439 - accuracy: 0.9391
Epoch 5/100
116/116 [==============================] - 0s 555us/step - loss: 0.1351 - accuracy: 0.9415
Epoch 6/100
116/116 [==============================] - 0s 467us/step - loss: 0.1298 - accuracy: 0.9431
Epoch 7/100
116/116 [==============================] - 0s 464us/step - loss: 0.1255 - accuracy: 0.9461
Epoch 8/100
116/116 [==============================] - 0s 461us/step - loss: 0.1224 - accuracy: 0.9488
Epoch 9/100
116/116 [==============================] - 0s 504us/step - loss: 0.1199 - accuracy: 0.9494
Epoch 10/100
116/116 [==============================] - 0s 467us/step - l

In [115]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

39/39 - 0s - loss: 0.1469 - accuracy: 0.9391 - 100ms/epoch - 3ms/step
Loss: 0.14693884551525116, Accuracy: 0.939073920249939
